In [2]:
import numpy as np
from py2neo import Graph, Node, Relationship
import pandas as pd
import graphistry

#### Workflow
* Pandas -> Neo4j API query (get nodes with given properties for example)
* Pandas <- Neo4j API query (query results to pandas, we can use the Neo4j features here)
* Pandas -> graphistry (visualiztaion, we can use different tools as well)
* Neo4j -> graphistry (?)


In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "monster"))

In [4]:
df_of_nodes = pd.read_csv("./Data/large_twitch_features.csv")
df_of_nodes.sort_values(by=["created_at"])

,views,mature,life_time,created_at,updated_at,numeric_id,dead_account,language,affiliate
137180,609,0,4158,2007-05-22,2018-10-09,137180,0,EN,0
82200,24316020,0,4161,2007-05-22,2018-10-12,82200,0,EN,0
49584,25965,0,4160,2007-05-22,2018-10-11,49584,0,SV,0
31140,10667564,0,4161,2007-05-22,2018-10-12,31140,0,EN,0
39212,3116,1,4155,2007-05-22,2018-10-06,39212,0,EN,0
...,...,...,...,...,...,...,...,...,...
24516,15,0,69,2018-05-12,2018-07-20,24516,0,EN,0
33361,125,0,119,2018-05-12,2018-09-08,33361,0,EN,0
4753,3281,1,150,2018-05-12,2018-10-09,4753,0,RU,1
57456,656,1,152,2018-05-13,2018-10-12,57456,0,EN,0


In [5]:
# dates without day give the monsth's first day 
query_for_given_date = """
    MATCH (u:user)
    WHERE date({year: 2007, month: 12}) > u.created_at >= date({year: 2007, month: 5, day: 22})
    RETURN u.numeric_id AS id, u.created_at AS CREATED_AT, u.language AS LANGUAGE 
    """

In [6]:
q_for_date_data = graph.run(query_for_given_date).to_data_frame()
q_for_date_data

,id,CREATED_AT,LANGUAGE
0,294,2007-05-22,EN
1,599,2007-07-13,EN
2,863,2007-10-07,EN
3,2750,2007-09-29,EN
4,3035,2007-10-08,EN
...,...,...,...
117,164770,2007-09-09,EN
118,164817,2007-08-25,EN
119,165474,2007-10-14,EN
120,165738,2007-11-08,EN


In [7]:
query_for_edges = """
    MATCH (u1:user)-[:FOLLOWS]-(u2:user)
    WHERE date({year: 2007, month: 12}) > u1.created_at >= date({year: 2007, month: 5, day: 22}) AND
         date({year: 2007, month: 12}) > u2.created_at >= date({year: 2007, month: 5, day: 22})
    RETURN u1.numeric_id AS id1, u2.numeric_id AS id2, u1.created_at AS CREATED_AT_1, 
    u2.created_at AS CREATED_AT_2
    """

In [8]:
q_for_date_edges = graph.run(query_for_edges).to_data_frame()
q_for_date_edges

,id1,id2,CREATED_AT_1,CREATED_AT_2
0,294,40326,2007-05-22,2007-05-22
1,599,40326,2007-07-13,2007-05-22
2,863,58773,2007-10-07,2007-06-28
3,4970,52703,2007-10-14,2007-05-22
4,6201,90697,2007-09-13,2007-10-25
...,...,...,...,...
191,162770,142634,2007-11-06,2007-05-29
192,163924,152996,2007-07-06,2007-07-05
193,165474,52703,2007-10-14,2007-05-22
194,165474,160946,2007-10-14,2007-06-29


In [9]:
q_for_date_edges.columns

Index(['id1', 'id2', 'CREATED_AT_1', 'CREATED_AT_2'], dtype='object')

In [10]:
q_for_date_new = q_for_date_edges[['id1', 'id2']]
q_for_date_new

,id1,id2
0,294,40326
1,599,40326
2,863,58773
3,4970,52703
4,6201,90697
...,...,...
191,162770,142634
192,163924,152996
193,165474,52703
194,165474,160946


In [13]:
graphistry.edges(q_for_date_new, 'id1', 'id2').plot()

In [232]:
df_of_edges = pd.read_csv("./Data/large_twitch_edges.csv")
df_of_edges.head(2)

,numeric_id_1,numeric_id_2
0,98343,141493
1,98343,58736


In [ ]:
graphistry.edges(q_for_date_new, 'id1', 'id2').plot()

In [222]:
g = graphistry.bind()
g = g.bind(source="id1", destination="id2", node='id').nodes(q_for_date_data).edges(q_for_date_edges)
g.plot()
"""
ig = g.pandas2igraph(q_for_date_edges)
g.plot(ig)"""

# g.plot()

NotImplementedError: 

In [215]:
?g

In [211]:
g = graphistry.bind()
g = g.bind(source="id1", destination="id2").edges(q_for_date_edges)
g.plot()

NotImplementedError: 

In [210]:
g = graphistry.bind()
g = g.bind(source="id1", destination="id2",node="id").nodes(q_for_date_data).edges(q_for_date_edges)
viz.plot()

NotImplementedError: 

In [187]:
# dates without day give the monsth's first day 
query_for_given_date = """
    MATCH (u:user {views: "7879"})
    RETURN u.numeric_id
    """

In [191]:
q_for_date_data = graph.run(query_for_given_date).to_data_frame()

In [192]:
q_for_date_data

,id,CREATED_AT,LANGUAGE
0,11,2014-01-31,EN


In [197]:
graph.run("""MATCH (u:user) WHERE u.mature="0" RETURN u.numeric_id""").evaluate()

'1'